<a href="https://colab.research.google.com/github/gdnsjf/educational-work/blob/main/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Используйте подход Нейронных Сетей (библиотека pytorch) и метод обратного распространения ошибки для данных находящихся по ссылке. Здесь - выходящей меткой (учителем) является столбец / атрибут - outcome.
Данный датасет мы рассматривали на лекции с одним внутренним слоем. Постройте топологию Нейронной Сети с
- 3 внутренними слоями (количество нейронов в сети пожалуйста подберите
сами) - 4 внутренними слоями (количество нейронов в сети пожалуйста подберите
сами)
Сравните значения метрики качества (accuracy) между указанными топологиями.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загрузка данных
data = pd.read_csv('dataset.csv')
# Предположим, что 'outcome' - это столбец с метками
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values
# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование данных в тензоры PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Создание датасетов и загрузчиков данных
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
class NeuralNet3Layers(nn.Module):
    def __init__(self):
        super(NeuralNet3Layers, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)  # Предполагаем, что у нас 2 класса в выходе

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class NeuralNet4Layers(nn.Module):
    def __init__(self):
        super(NeuralNet4Layers, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 2)  # Предполагаем, что у нас 2 класса в выходе

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x
def train(model, train_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return correct / total

# Инициализация моделей, критериев и оптимизаторов
model3 = NeuralNet3Layers()
model4 = NeuralNet4Layers()

criterion = nn.CrossEntropyLoss()
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)
optimizer4 = optim.Adam(model4.parameters(), lr=0.001)

# Обучение моделей
train(model3, train_loader, criterion, optimizer3, epochs=20)
train(model4, train_loader, criterion, optimizer4, epochs=20)

# Оценка моделей
accuracy3 = evaluate(model3, test_loader)
accuracy4 = evaluate(model4, test_loader)

print(f'Accuracy of the model with 3 layers: {accuracy3 * 100:.2f}%')
print(f'Accuracy of the model with 4 layers: {accuracy4 * 100:.2f}%')

Accuracy of the model with 3 layers: 74.03%
Accuracy of the model with 4 layers: 68.18%
